# Part II: Model Development

In this part, we develop three unique pipelines for predicting backorder. We use the smart sample from Part I to fit and evaluate these pipelines. 

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

import os, sys
import itertools
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_score, train_test_split

## Reload the smart sample here

In [2]:

#import pickle
import joblib

#load the dataset
dataset= joblib.load('datasetnew.pkl')


In [3]:
type(dataset)

pandas.core.frame.DataFrame

In [4]:
dataset.head(10)

,national_inv,lead_time,in_transit_qty,forecast_3_month,forecast_6_month,forecast_9_month,sales_1_month,sales_3_month,sales_6_month,sales_9_month,...,pieces_past_due,perf_6_month_avg,perf_12_month_avg,local_bo_qty,deck_risk,oe_constraint,ppap_risk,stop_auto_buy,rev_stop,went_on_backorder
34,0.0,12.0,0.0,48.0,84.0,120.0,7.0,45.0,81.0,130.0,...,0.0,0.98,0.89,0.0,0,0,0,1,0,1
399,5.0,4.0,0.0,1.0,3.0,6.0,0.0,4.0,7.0,9.0,...,0.0,0.89,0.90,0.0,0,0,0,1,0,1
834,0.0,8.0,0.0,8.0,8.0,12.0,1.0,2.0,6.0,6.0,...,0.0,0.78,0.78,0.0,0,0,0,1,0,1
1099,0.0,8.0,2.0,12.0,17.0,27.0,8.0,17.0,28.0,38.0,...,0.0,0.42,0.36,0.0,0,0,0,1,0,1
1115,6.0,8.0,0.0,19.0,19.0,30.0,4.0,15.0,32.0,61.0,...,18.0,0.18,0.31,0.0,0,0,0,1,0,1
1289,49.0,8.0,215.0,1000.0,1800.0,2400.0,481.0,884.0,1358.0,2116.0,...,200.0,0.00,0.00,6.0,0,0,1,1,0,1
1363,7.0,2.0,1.0,0.0,0.0,0.0,2.0,17.0,23.0,23.0,...,0.0,0.99,0.90,0.0,1,0,0,1,0,1
1391,0.0,4.0,0.0,168.0,168.0,168.0,10.0,36.0,55.0,81.0,...,0.0,0.84,0.86,0.0,0,0,0,1,0,1
1436,13.0,8.0,0.0,100.0,100.0,100.0,5.0,25.0,47.0,65.0,...,0.0,0.97,0.96,0.0,0,0,0,1,0,1
1644,0.0,8.0,0.0,28.0,56.0,70.0,7.0,17.0,26.0,39.0,...,0.0,0.98,0.97,0.0,0,0,0,1,0,1


In [5]:
#Exploratory data analysis to find requirement of normalization/standarization

dataset.describe().transpose()


,count,mean,std,min,25%,50%,75%,max
national_inv,22586.0,245.081555,9465.405980,-2999.0,1.00,5.00,24.00,1370327.0
lead_time,22586.0,7.178252,5.811139,0.0,2.00,8.00,8.00,52.0
in_transit_qty,22586.0,26.680643,715.753351,0.0,0.00,0.00,0.00,81720.0
forecast_3_month,22586.0,163.835562,2448.302626,0.0,0.00,3.00,24.00,252790.0
forecast_6_month,22586.0,289.543390,4622.241465,0.0,0.00,6.00,41.00,495890.0
forecast_9_month,22586.0,406.808288,6602.215436,0.0,0.00,9.00,60.00,719270.0
sales_1_month,22586.0,39.904587,607.188377,0.0,0.00,1.00,7.00,71757.0
sales_3_month,22586.0,119.580625,1855.757475,0.0,0.00,4.00,21.00,226570.0
sales_6_month,22586.0,224.919729,3661.144705,0.0,0.00,8.00,39.00,450657.0
sales_9_month,22586.0,339.190295,5641.342220,0.0,1.00,11.00,57.00,701532.0


## Split the data into Train/Test

In [6]:
X = np.array(dataset.iloc[:,0:21])
y = np.array(dataset.went_on_backorder)
print(X.shape)
print(y.shape)

#70% data on training and 20% data on tesing set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
print("Training shapes (X, y): ", X_train.shape, y_train.shape)
print("Testing shapes (X, y): ", X_test.shape, y_test.shape)

(22586, 21)
(22586,)
Training shapes (X, y):  (15810, 21) (15810,)
Testing shapes (X, y):  (6776, 21) (6776,)


## Developing Pipeline

In this section, we design an operationalized machine learning pipeline, which includes:

* Anomaly detection
* Dimensionality Reduction
* Train a classification model

In [7]:
#libraries

from sklearn.svm import OneClassSVM
from sklearn.neighbors import LocalOutlierFactor
from sklearn.covariance import EllipticEnvelope
from sklearn.ensemble import IsolationForest

from sklearn.decomposition import PCA, FactorAnalysis
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, chi2, f_classif, mutual_info_classif

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.neighbors import NearestNeighbors
from sklearn.feature_selection import RFE
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import IsolationForest
from sklearn.svm import OneClassSVM
from sklearn.feature_selection import VarianceThreshold
from sklearn.neighbors import LocalOutlierFactor

### Pipeline-1 
  * Anomaly detection
  * Dimensionality reduction
  * Model training/validation

In [1]:
# Add anomaly detection code  (Question #E201)
# ----------------------------------
# Anomaly detection method: EllipticEnvelope
envelope = EllipticEnvelope(support_fraction=1, contamination=0.2).fit(X_train)

# Create an boolean indexing array to pick up outliers
outliers = envelope.predict(X_train)==-1

# Re-slice X train,y train into a cleaned dataset with outliers excluded
X_ee_train = X_train[~outliers]
y_ee_train = y_train[~outliers]
print(X_ee_train.shape)
print(y_ee_train.shape)

NameError: name 'EllipticEnvelope' is not defined

In [9]:
# Add codes for feature selection and classification pipeline with grid search  (Question #E202)
# ----------------------------------

# Feature selection: PCA  Classifier: svc/ kernel SVC
#model with outliers on test set
#model1
#defining pipeline
pipe = Pipeline([
    ('scale', MinMaxScaler()),
    ('PCA', PCA()), 
    ('SVC', SVC())  
])

#hyperparameter tuning for pca and svc
param_grid = {'PCA__n_components': [20],
              'SVC__C': np.linspace(1000, 10000, num=4), 
              'SVC__gamma': np.linspace(0.1, 0.5, num=4), 
              'SVC__kernel': ['rbf']}


#model building and fitting model on train set
model1 = GridSearchCV(pipe, param_grid, cv=5, n_jobs=-1)
model1.fit(X_ee_train, y_ee_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scale', MinMaxScaler()),
                                       ('PCA', PCA()), ('SVC', SVC())]),
             n_jobs=-1,
             param_grid={'PCA__n_components': [20],
                         'SVC__C': array([ 1000.,  4000.,  7000., 10000.]),
                         'SVC__gamma': array([0.1       , 0.23333333, 0.36666667, 0.5       ]),
                         'SVC__kernel': ['rbf']})

In [10]:
#model accuracy
model_results2 = pd.DataFrame.from_dict(model1.cv_results_)
print(model_results2.sort_values("rank_test_score"))

    mean_fit_time  std_fit_time  mean_score_time  std_score_time  \
11      27.401864      0.922744         0.753493        0.007899   
6       16.279418      0.476982         0.790026        0.012372   
15      36.402536      1.081011         0.795253        0.066289   
14      31.260587      1.416480         0.761762        0.009342   
13      23.661367      0.891306         0.866684        0.098478   
7       18.559729      0.523359         0.776303        0.026145   
10      22.148836      0.767508         0.768409        0.009063   
9       18.591622      0.618538         0.834388        0.060129   
5       13.461620      0.548058         0.853715        0.044101   
8       13.344809      0.510437         0.861897        0.008616   
2        8.446556      0.147409         0.832017        0.011122   
3        8.796750      0.171998         0.808840        0.011039   
12      15.948847      0.816654         0.866792        0.028473   
4       10.266752      0.400289         0.905704

In [11]:
# Given an unbiased evaluation  (Question #E203)
# ----------------------------------
#model with outliers on test set
#model building and fitting model on train set
print("Parameters of the best model are:",model1.best_estimator_)
print("Score of the best model:", model1.best_score_)
bestmodel1 = model1.best_estimator_
predicted_y = model1.predict(X_test)
print(classification_report(y_test, predicted_y))
print("Confusion Matrix")
pd.DataFrame(confusion_matrix(y_test, predicted_y))

Parameters of the best model are: Pipeline(steps=[('scale', MinMaxScaler()), ('PCA', PCA(n_components=20)),
                ('SVC', SVC(C=7000.0, gamma=0.5))])
Score of the best model: 0.8568157515117132
              precision    recall  f1-score   support

           0       0.80      0.86      0.83      3402
           1       0.84      0.79      0.81      3374

    accuracy                           0.82      6776
   macro avg       0.82      0.82      0.82      6776
weighted avg       0.82      0.82      0.82      6776

Confusion Matrix


,0,1
0,2912,490
1,722,2652


In [12]:
#model after ourlier reduction on test data set

#Anomaly detection and reduction on test set
# Anomaly detection method: EllipticEnvelope
envelope = EllipticEnvelope(support_fraction=1, contamination=0.2).fit(X_test)

# Create an boolean indexing array to pick up outliers
outliers = envelope.predict(X_test)==-1

# Re-slice X train,y train into a cleaned dataset with outliers excluded
X_ee_test = X_test[~outliers]
y_ee_test = y_test[~outliers]


#model building and fitting model on train set

predicted_y = model1.predict(X_ee_test)
print(classification_report(y_ee_test, predicted_y))
print("Confusion Matrix")
pd.DataFrame(confusion_matrix(y_ee_test, predicted_y))

              precision    recall  f1-score   support

           0       0.87      0.85      0.86      2675
           1       0.85      0.88      0.87      2745

    accuracy                           0.86      5420
   macro avg       0.86      0.86      0.86      5420
weighted avg       0.86      0.86      0.86      5420

Confusion Matrix


,0,1
0,2266,409
1,338,2407


In [13]:
# Given an unbiased evaluation  (Question #E203)
# ----------------------------------
#another way of model1

# Feature Selection: PCA , Classifier: SVC/ Kernel SVC
# Pipeline defination
pipe = Pipeline([
    ('scale', MinMaxScaler()),
    ('PCA', PCA()), 
    ('SVC', SVC())  
])

# hyperparameter tuning of classifiers and feature selection method

param_grid = {'SVC__C': [500, 1000, 5000, 10000], 
              'SVC__gamma': [0.1, 0.5], 
              'PCA__n_components': [10, 20],
              'SVC__kernel': ['rbf']}


model1n = GridSearchCV(pipe, param_grid, cv=5, n_jobs=-1)
model1n.fit(X_ee_train, y_ee_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scale', MinMaxScaler()),
                                       ('PCA', PCA()), ('SVC', SVC())]),
             n_jobs=-1,
             param_grid={'PCA__n_components': [10, 20],
                         'SVC__C': [500, 1000, 5000, 10000],
                         'SVC__gamma': [0.1, 0.5], 'SVC__kernel': ['rbf']})

In [14]:
#model accuracy
model_results2 = pd.DataFrame.from_dict(model1n.cv_results_)
print(model_results2.sort_values("rank_test_score"))

    mean_fit_time  std_fit_time  mean_score_time  std_score_time  \
13      21.405757      0.903835         0.766710        0.016909   
15      35.882653      1.397312         0.742288        0.018743   
11       8.535677      0.144737         0.814909        0.019983   
14      15.564561      0.802227         0.858056        0.009200   
12      11.298419      0.303327         0.882565        0.013260   
9        7.027674      0.128893         0.858346        0.024250   
7       32.556818      1.312969         0.671891        0.011544   
10       7.000104      0.493210         0.923907        0.011336   
8        6.468229      0.468659         1.003018        0.096377   
5       19.040603      0.922520         0.696299        0.010870   
3        8.708659      0.300155         0.734626        0.040387   
1        7.220804      0.215060         0.762125        0.020741   
6       14.684410      0.516697         0.786570        0.017308   
4       10.723770      0.355259         0.785646

In [15]:
#confusion matrix, precision, recall, f1-score, accuracy

print("Parameters of the best model are:",model1n.best_estimator_)
print("Score of the best model:", model1n.best_score_)
predicted_y = model1n.predict(X_test)
print(classification_report(y_test, predicted_y))
pd.DataFrame(confusion_matrix(y_test, predicted_y))

Parameters of the best model are: Pipeline(steps=[('scale', MinMaxScaler()), ('PCA', PCA(n_components=20)),
                ('SVC', SVC(C=5000, gamma=0.5))])
Score of the best model: 0.8569738230205506
              precision    recall  f1-score   support

           0       0.80      0.86      0.83      3402
           1       0.85      0.78      0.81      3374

    accuracy                           0.82      6776
   macro avg       0.82      0.82      0.82      6776
weighted avg       0.82      0.82      0.82      6776



,0,1
0,2920,482
1,728,2646


#### <center>Record the optimal hyperparameters and performance resulting from this pipeline.</center>

## <span style="background: yellow;">Commit your code!</span> 

### Your 2nd pipeline
  * Anomaly detection
  * Dimensionality reduction
  * Model training/validation

In [16]:
# Add anomaly detection code  (Question #E205)
# ----------------------------------
#model2 with outliers on test set
# Anomaly detection method: IsolationForest 
iso_forest = IsolationForest(contamination=0.05).fit(X_train, y_train)

iso_outliers = iso_forest.predict(X_train)==-1

X_iso_train = X_train[~iso_outliers]
y_iso_train = y_train[~iso_outliers]

print(X_iso_train.shape)
print(y_iso_train.shape)

(15019, 21)
(15019,)


In [17]:
# Add codes for feature selection and classification pipeline with grid search  (Question #E206)
# ----------------------------------
#model2
#feature selection method: Factor Analysis and classifier: LinearSVC
#pipeline defination
pipe = Pipeline([
    ('scale', StandardScaler()),
    ('FA', FactorAnalysis()),
    ('LinearSVC', LinearSVC())                 
])

# hyperparameter tunning of the classifier and feature selection method
param_grid = {'FA__n_components': [15, 20],
              'LinearSVC__C': [10, 20, 30, 40, 50],
              'LinearSVC__penalty': ['l1', 'l2']
              }


# building and fitting model on train set
model2 = GridSearchCV(pipe, param_grid, cv=5, n_jobs=-1)
model2.fit(X_iso_train, y_iso_train)


/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_search.py:921: UserWarning: One or more of the test scores are non-finite: [       nan 0.70091211        nan 0.69725146        nan 0.69845013
        nan 0.69685082        nan 0.69878291        nan 0.73200605
        nan 0.73779842        nan 0.74152703        nan 0.7514478
        nan 0.73260401]
  category=UserWarning
/opt/conda/lib/python3.7/site-packages/sklearn/svm/_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scale', StandardScaler()),
                                       ('FA', FactorAnalysis()),
                                       ('LinearSVC', LinearSVC())]),
             n_jobs=-1,
             param_grid={'FA__n_components': [15, 20],
                         'LinearSVC__C': [10, 20, 30, 40, 50],
                         'LinearSVC__penalty': ['l1', 'l2']})

In [18]:
#model accuracy
model_results2 = pd.DataFrame.from_dict(model2.cv_results_)
print(model_results2.sort_values("rank_test_score"))

    mean_fit_time  std_fit_time  mean_score_time  std_score_time  \
17       2.936180      0.147945         0.002229        0.000045   
15       2.965025      0.126044         0.002250        0.000064   
13       2.924571      0.101103         0.002329        0.000075   
19       2.679536      0.193225         0.002202        0.000074   
11       2.788662      0.107845         0.002305        0.000134   
1        4.157096      0.775368         0.002215        0.000132   
9        4.011814      0.754822         0.002172        0.000020   
5        3.930265      0.656304         0.002195        0.000094   
3        4.062073      0.673041         0.002234        0.000160   
7        4.011479      0.683250         0.002192        0.000120   
8        2.744943      0.670006         0.000000        0.000000   
18       1.303190      0.115428         0.000000        0.000000   
10       1.338654      0.113754         0.000000        0.000000   
12       1.303090      0.079143         0.000000

In [19]:
# Given an unbiased evaluation  (Question #E207)
# ----------------------------------

#confusion matrix, precision, recall, f1-score, accuracy
print("Parameters of the best model are:",model2.best_estimator_)
print("Score of the best model:", model2.best_score_)
bestmodel2 = model2.best_estimator_
predicted_y = model2.predict(X_test)
print(classification_report(y_test, predicted_y))
pd.DataFrame(confusion_matrix(y_test, predicted_y))

Parameters of the best model are: Pipeline(steps=[('scale', StandardScaler()),
                ('FA', FactorAnalysis(n_components=20)),
                ('LinearSVC', LinearSVC(C=40))])
Score of the best model: 0.7514477976528575
              precision    recall  f1-score   support

           0       0.83      0.60      0.70      3402
           1       0.68      0.88      0.77      3374

    accuracy                           0.74      6776
   macro avg       0.76      0.74      0.73      6776
weighted avg       0.76      0.74      0.73      6776



,0,1
0,2034,1368
1,412,2962


In [20]:
#model2 with only inliers on test set

#model2 with outliers on test set
# Anomaly detection method: IsolationForest 
iso_forest = IsolationForest(contamination=0.05).fit(X_test, y_test)

iso_outliers = iso_forest.predict(X_test)==-1

X_iso_test = X_test[~iso_outliers]
y_iso_test = y_test[~iso_outliers]

#confusion matrix, precision, recall, f1-score, accuracy

predicted_y = model2.predict(X_iso_test)
print(classification_report(y_iso_test, predicted_y))
pd.DataFrame(confusion_matrix(y_iso_test, predicted_y))

              precision    recall  f1-score   support

           0       0.83      0.60      0.70      3224
           1       0.69      0.88      0.77      3213

    accuracy                           0.74      6437
   macro avg       0.76      0.74      0.73      6437
weighted avg       0.76      0.74      0.73      6437



,0,1
0,1929,1295
1,394,2819


#### <center>Record the optimal hyperparameters and performance resulting from this pipeline.</center>

## <span style="background: yellow;">Commit your code!</span> 

### Your 3rd pipeline
  * Anomaly detection
  * Dimensionality reduction
  * Model training/validation

In [21]:
# Add anomaly detection code  (Question #E209)
# ----------------------------------

#Anomaly detection method:Local outlier factor
lof_model = LocalOutlierFactor(n_neighbors=20)
lof=lof_model.fit(X_train)
train_outliers = lof_model.fit_predict(X_train)
X_lof_train = X_train[train_outliers == 1]
y_lof_train = y_train[train_outliers == 1]


In [22]:
# Add codes for feature selection and classification pipeline with grid search  (Question #E210)
# ----------------------------------
#model 3
#model with outlier ontrain set
#feature selection method: factor analysis, Classifier: random forest
#defining pipeline
pipeline = Pipeline([
    ('scale', StandardScaler()),
    ('RFE', RFE(estimator=RandomForestClassifier())),
    ('rf', RandomForestClassifier())
])

# define a grid of hyperparameters for the pipeline
param_grid = {
     'RFE__n_features_to_select': [10, 15],
     'rf__n_estimators': [50, 100, 1000],
     'rf__min_samples_split': [2, 3, 4],
     'rf__criterion': ['gini', 'entropy'] 
}

#model building and fitting model on train set
model3 = GridSearchCV(pipeline, param_grid, cv=5, n_jobs=-1)
model3.fit(X_lof_train, y_lof_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scale', StandardScaler()),
                                       ('RFE',
                                        RFE(estimator=RandomForestClassifier())),
                                       ('rf', RandomForestClassifier())]),
             n_jobs=-1,
             param_grid={'RFE__n_features_to_select': [10, 15],
                         'rf__criterion': ['gini', 'entropy'],
                         'rf__min_samples_split': [2, 3, 4],
                         'rf__n_estimators': [50, 100, 1000]})

In [23]:
#model accuracy
model_results2 = pd.DataFrame.from_dict(model3.cv_results_)
print(model_results2.sort_values("rank_test_score"))

    mean_fit_time  std_fit_time  mean_score_time  std_score_time  \
29      26.256861      0.516521         0.677904        0.020587   
19      10.887834      0.045051         0.064631        0.001502   
28      11.256159      0.036070         0.065990        0.000699   
32      25.975243      0.214289         0.660366        0.008512   
20      22.349998      0.126186         0.674122        0.015259   
35      26.080022      0.201758         0.668486        0.063980   
23      22.291959      0.173757         0.662475        0.006202   
18      10.288435      0.035826         0.033979        0.001248   
31      11.275072      0.022975         0.065635        0.000489   
22      11.010592      0.071385         0.064152        0.001601   
33      10.506011      0.058887         0.034847        0.003692   
27      10.429851      0.042954         0.033868        0.002090   
26      22.212767      0.104337         0.644953        0.005552   
34      11.480753      0.124960         0.064747

In [24]:
# Given an unbiased evaluation  (Question #E211)
# ----------------------------------
#confusion matrix, precision, recall, f1-score, accuracy

print("Parameters of the best model are:",model3.best_estimator_)
print("Score of the best model:", model3.best_score_)
bestmodel3 = model3.best_estimator_
predicted_y = model3.predict(X_test)
print(classification_report(y_test, predicted_y))
pd.DataFrame(confusion_matrix(y_test, predicted_y))

Parameters of the best model are: Pipeline(steps=[('scale', StandardScaler()),
                ('RFE',
                 RFE(estimator=RandomForestClassifier(),
                     n_features_to_select=15)),
                ('rf',
                 RandomForestClassifier(criterion='entropy',
                                        n_estimators=1000))])
Score of the best model: 0.9093877036397359
              precision    recall  f1-score   support

           0       0.93      0.89      0.91      3402
           1       0.89      0.93      0.91      3374

    accuracy                           0.91      6776
   macro avg       0.91      0.91      0.91      6776
weighted avg       0.91      0.91      0.91      6776



,0,1
0,3029,373
1,233,3141


In [25]:
#model without outliers on test data

#Anomaly detection method:Local outlier factor
lof_model = LocalOutlierFactor(n_neighbors=20)
lof_model.fit(X_test)
test_outliers = lof_model.fit_predict(X_test)
X_lof_test = X_test[test_outliers == 1]
y_lof_test = y_test[test_outliers == 1]


#confusion matrix, precision, recall, f1-score, accuracy

predicted_y = model3.predict(X_lof_test)
print(classification_report(y_lof_test, predicted_y))
pd.DataFrame(confusion_matrix(y_lof_test, predicted_y))

              precision    recall  f1-score   support

           0       0.93      0.89      0.91      2968
           1       0.90      0.94      0.92      3091

    accuracy                           0.91      6059
   macro avg       0.92      0.91      0.91      6059
weighted avg       0.92      0.91      0.91      6059



,0,1
0,2643,325
1,191,2900


#### <center>Record the optimal hyperparameters and performance resulting from this pipeline.</center>

##### Trial and practice(to finalize the model)

In [78]:
#Anomaly detection method: OneClassSVM
# Construct OneClassSVM 
svm = OneClassSVM(kernel='rbf').fit(X_train, y_train)

# Get labels from classifier and cull outliers
svm_outliers = svm.predict(X_train)==-1

X_svm = X_train[~svm_outliers]
y_svm = y_train[~svm_outliers]

In [79]:
#model4
#Anomaly Detection:OneclassSVM, feature selection: RFE, Classifier: LinearSVC

#pipeline defination
pipe = Pipeline([
    ('scale', StandardScaler()),
    ('RFE', RFE(estimator=LinearSVC())),
    ('LinearSVC', LinearSVC())                 
])

# hyperparameter tunning of the classifier and feature selection method
param_grid = {'RFE__n_features_to_select': [5, 10, 15],
              'LinearSVC__C': [10, 20, 30, 40, 50],
              'LinearSVC__penalty': ['l2']
              }


# building and fitting model on train set
model4 = GridSearchCV(pipe, param_grid, cv=5, n_jobs=-1)
model4.fit(X_svm, y_svm)
#confusion matrix, precision, recall, f1-score, accuracy
print("Parameters of the best model are:",model4.best_estimator_)
print("Score of the best model:", model3.best_score_)
predicted_y = model4.predict(X_test)
print(classification_report(y_test, predicted_y))
pd.DataFrame(confusion_matrix(y_test, predicted_y))
pd.DataFrame(confusion_matrix(y_test, predicted_y))


/opt/conda/lib/python3.7/site-packages/sklearn/svm/_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarn

Parameters of the best model are: Pipeline(steps=[('scale', StandardScaler()),
                ('RFE', RFE(estimator=LinearSVC(), n_features_to_select=5)),
                ('LinearSVC', LinearSVC(C=40))])
Score of the best model: 0.9061216551941215
              precision    recall  f1-score   support

           0       0.94      0.43      0.59      3395
           1       0.63      0.97      0.77      3381

    accuracy                           0.70      6776
   macro avg       0.79      0.70      0.68      6776
weighted avg       0.79      0.70      0.68      6776



/opt/conda/lib/python3.7/site-packages/sklearn/svm/_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


,0,1
0,1471,1924
1,94,3287


In [80]:
#model 5
#Anomaly detection: OneclassSVM ,feature selection: RFE, Classifier: Logistic regression
#pipeline defination
pipe = Pipeline([
    ('scale', StandardScaler()),
    ('RFE', RFE(estimator=LogisticRegression())),
    ('LogisticRegression', LogisticRegression())                 
])

#hyperparameter tunning for the classifier and feature selection method
param_grid = { 'RFE__n_features_to_select': [10, 15],
               'LogisticRegression__C': [4, 5, 6, 9, 10]
              }

# model building and fitting on training set
model5 = GridSearchCV(pipe, param_grid, cv=5, n_jobs=-1)
model5.fit(X_svm, y_svm)

# Given an unbiased evaluation  (Question #E211)
# ----------------------------------
#confusion matrix, precision, recall, f1-score, accuracy
print("Parameters of the best model are:",model5.best_estimator_)
print("Score of the best model:", model5.best_score_)
predicted_y = model5.predict(X_test)
print(classification_report(y_test, predicted_y))
pd.DataFrame(confusion_matrix(y_test, predicted_y))
pd.DataFrame(confusion_matrix(y_test, predicted_y))

Parameters of the best model are: Pipeline(steps=[('scale', StandardScaler()),
                ('RFE',
                 RFE(estimator=LogisticRegression(), n_features_to_select=10)),
                ('LogisticRegression', LogisticRegression(C=5))])
Score of the best model: 0.8527514231499052
              precision    recall  f1-score   support

           0       0.94      0.46      0.62      3395
           1       0.64      0.97      0.77      3381

    accuracy                           0.72      6776
   macro avg       0.79      0.72      0.70      6776
weighted avg       0.79      0.72      0.70      6776



,0,1
0,1572,1823
1,98,3283


In [81]:
# Anamoly Detection Method: SGDOneClassSVM

clf = OneClassSVM(nu=0.1, kernel='linear', gamma=0.1)

# Fit the model to the training data
clf.fit(X_train)

# Predict inliers/outliers
y_pred_train = clf.predict(X_train)

# Identify and remove outliers
mask = y_pred_train == 1
X_svml = X_train[mask]
y_svml = y_train[mask]

In [82]:
#model6
#Anomaly detection: SGDOneClassSVM, feature selection methos: Variance Threshold, Classifier: Logistic regression
#defining pipeline
pipe = Pipeline([
    ('scale', StandardScaler()),
    ('VT', VarianceThreshold()),
    ('LogisticRegression', LogisticRegression())                 
])
#hyperparameter tunning for feature selection method and classifier
param_grid = {'VT__threshold': [0.0],
               'LogisticRegression__C': [4, 5, 6, 9, 10]
              }

#model building and fitting
model6 = GridSearchCV(pipe, param_grid, cv=5, n_jobs=-1)
model6.fit(X_svml, y_svml)


# Given an unbiased evaluation  (Question #E211)
# ----------------------------------
#confusion matrix, precision, recall, f1-score, accuracy
print("Parameters of the best model are:",model6.best_estimator_)
print("Score of the best model:", model6.best_score_)
predicted_y = model6.predict(X_test)
print(classification_report(y_test, predicted_y))
pd.DataFrame(confusion_matrix(y_test, predicted_y))
pd.DataFrame(confusion_matrix(y_test, predicted_y))

Parameters of the best model are: Pipeline(steps=[('scale', StandardScaler()), ('VT', VarianceThreshold()),
                ('LogisticRegression', LogisticRegression(C=10))])
Score of the best model: 0.6808405964280025
              precision    recall  f1-score   support

           0       0.77      0.46      0.58      3395
           1       0.61      0.86      0.72      3381

    accuracy                           0.66      6776
   macro avg       0.69      0.66      0.65      6776
weighted avg       0.69      0.66      0.65      6776



/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


,0,1
0,1565,1830
1,458,2923


In [83]:
#model 7
#Anomaly detection: SGDOneClassSVM, feature selection method: RFE, Classifier: logistic regression
#pipeline defination
pipe = Pipeline([
    ('scale', StandardScaler()),
    ('RFE', RFE(estimator=LogisticRegression())),
    ('LogisticRegression', LogisticRegression())                 
])
# hyperparameter tunning for classifier and feature selector
param_grid = { 'RFE__n_features_to_select': [10, 15],
               'LogisticRegression__C': [4, 5, 6, 9, 10]
              }

#model building and model fitting on train data
model7 = GridSearchCV(pipe, param_grid, cv=5, n_jobs=-1)
model7.fit(X_svml, y_svml)


# Given an unbiased evaluation  (Question #E211)
# ----------------------------------
#confusion matrix, precision, recall, f1-score, accuracy
print("Parameters of the best model are:",model7.best_estimator_)
print("Score of the best model:", model7.best_score_)
predicted_y = model7.predict(X_test)
print(classification_report(y_test, predicted_y))
pd.DataFrame(confusion_matrix(y_test, predicted_y))

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


Parameters of the best model are: Pipeline(steps=[('scale', StandardScaler()),
                ('RFE',
                 RFE(estimator=LogisticRegression(), n_features_to_select=10)),
                ('LogisticRegression', LogisticRegression(C=10))])
Score of the best model: 0.6799968876862293
              precision    recall  f1-score   support

           0       0.78      0.43      0.55      3395
           1       0.60      0.88      0.72      3381

    accuracy                           0.65      6776
   macro avg       0.69      0.65      0.63      6776
weighted avg       0.69      0.65      0.63      6776



/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


,0,1
0,1456,1939
1,418,2963


In [ ]:
#model8

#Anomaly detection: isolation forest, feature selection method: RFE, Classifier: random forest
#defining pipeline
pipeline = Pipeline([
    ('scale', StandardScaler()),
    ('RFE', RFE(estimator=RandomForestClassifier()))
    ('rf', RandomForestClassifier())
])

# define a grid of hyperparameters for the pipeline
param_grid = {
     'RFE__n_features_to_select': [10, 15],
     'rf__n_estimators': [50, 100, 1000],
     'rf__min_samples_split': [2, 3, 4],
     'rf__criterion': ['gini', 'entropy'] 
}

#model building and fitting model on train set
model8 = GridSearchCV(pipeline, param_grid, cv=5, n_jobs=-1)
model8.fit(X_iso, y_iso)


#confusion matrix, precision, recall, f1-score, accuracy

print("Parameters of the best model are:",model8.best_estimator_)
print("Score of the best model:", model8.best_score_)
predicted_y = model8.predict(X_test)
print(classification_report(y_test, predicted_y))
pd.DataFrame(confusion_matrix(y_test, predicted_y))

In [85]:
#model 9
#Anomaly detection: EllipticEnvelope, feature selection: Factor Analysis, classifier: Random Forest
pipeline = Pipeline([
    ('scale', StandardScaler()),
    ('feature_selection', FactorAnalysis()),
    ('rf', RandomForestClassifier())
])

# define a grid of hyperparameters for the pipeline
param_grid = {
     'feature_selection__n_components': [15, 20],
     'rf__n_estimators': [50, 100, 1000],
     'rf__min_samples_split': [2, 3, 4],
     'rf__criterion': ['gini', 'entropy'] 
}

#model building and fitting model on train set
model9 = GridSearchCV(pipeline, param_grid, cv=5, n_jobs=-1)
model9.fit(X_ee, y_ee)


#confusion matrix, precision, recall, f1-score, accuracy

print("Parameters of the best model are:",model9.best_estimator_)
print("Score of the best model:", model9.best_score_)
predicted_y = model9.predict(X_test)
print(classification_report(y_test, predicted_y))
pd.DataFrame(confusion_matrix(y_test, predicted_y))

Parameters of the best model are: Pipeline(steps=[('scale', StandardScaler()),
                ('feature_selection', FactorAnalysis(n_components=20)),
                ('rf', RandomForestClassifier(n_estimators=1000))])
Score of the best model: 0.8617964575352784
              precision    recall  f1-score   support

           0       0.88      0.81      0.84      3395
           1       0.82      0.89      0.85      3381

    accuracy                           0.85      6776
   macro avg       0.85      0.85      0.85      6776
weighted avg       0.85      0.85      0.85      6776



,0,1
0,2739,656
1,369,3012


In [10]:
#model10
#Anomaly detection: local outlier factor, feature selection method: factor analysis, Classifier: random forest
#defining pipeline
pipeline = Pipeline([
    ('scale', StandardScaler()),
    ('RFE', RFE(estimator=RandomForestClassifier())),
    ('rf', RandomForestClassifier())
])

# define a grid of hyperparameters for the pipeline
param_grid = {
     'RFE__n_features_to_select': [10, 15],
     'rf__n_estimators': [50, 100, 1000],
     'rf__min_samples_split': [2, 3, 4],
     'rf__criterion': ['gini', 'entropy'] 
}

#model building and fitting model on train set
model10 = GridSearchCV(pipeline, param_grid, cv=5, n_jobs=-1)
model10.fit(X_lof, y_lof)


#confusion matrix, precision, recall, f1-score, accuracy

print("Parameters of the best model are:",model10.best_estimator_)
print("Score of the best model:", model10.best_score_)
predicted_y = model10.predict(X_test)
print(classification_report(y_test, predicted_y))
pd.DataFrame(confusion_matrix(y_test, predicted_y))

Parameters of the best model are: Pipeline(steps=[('scale', StandardScaler()),
                ('RFE',
                 RFE(estimator=RandomForestClassifier(),
                     n_features_to_select=15)),
                ('rf',
                 RandomForestClassifier(criterion='entropy',
                                        n_estimators=1000))])
Score of the best model: 0.9098608965604734
              precision    recall  f1-score   support

           0       0.93      0.89      0.91      3447
           1       0.89      0.93      0.91      3329

    accuracy                           0.91      6776
   macro avg       0.91      0.91      0.91      6776
weighted avg       0.91      0.91      0.91      6776



,0,1
0,3068,379
1,219,3110


In [11]:
#model 11
#Anomaly Detection: EllipticEnvelope, feature selection: Factor Analysis, Classifier: kernel SVC 
pipe = Pipeline([
    ('scale', MinMaxScaler()),
    ('FA', FactorAnalysis()), 
    ('SVC', SVC())  
])

# hyperparameter tuning of classifiers and feature selection method

param_grid = {
              'FA__n_components': [15, 20],
              'SVC__C': [500, 1000, 5000, 10000], 
              'SVC__gamma': [0.1, 0.5], 
              'SVC__kernel': ['rbf']
          }


model10 = GridSearchCV(pipe, param_grid, cv=5, n_jobs=-1)
model10.fit(X_ee, y_ee)

print("Parameters of the best model are:",model110.best_estimator_)
print("Score of the best model:", model10.best_score_)
predicted_y = model110.predict(X_test)
print(classification_report(y_test, predicted_y))
pd.DataFrame(confusion_matrix(y_test, predicted_y))

Parameters of the best model are: Pipeline(steps=[('scale', MinMaxScaler()),
                ('FA', FactorAnalysis(n_components=20)),
                ('SVC', SVC(C=500, gamma=0.1))])
Score of the best model: 0.8453513316672842
              precision    recall  f1-score   support

           0       0.76      0.88      0.82      3387
           1       0.86      0.73      0.79      3389

    accuracy                           0.80      6776
   macro avg       0.81      0.80      0.80      6776
weighted avg       0.81      0.80      0.80      6776



,0,1
0,2973,414
1,925,2464


In [12]:
#model 12
#Anomaly detection: Isolation Forest, feature selection: PCA, classification: Logistic Regression
pipe = Pipeline([
    ('scale', StandardScaler()),
    ('PCA', PCA()),
    ('LogisticRegression', LogisticRegression())                 
])
# hyperparameter tunning for classifier and feature selector
param_grid = { 'PCA__n_components': [15, 20],
               'LogisticRegression__C': [4, 5, 6, 9, 10]
              }

#model building and model fitting on train data
model12 = GridSearchCV(pipe, param_grid, cv=5, n_jobs=-1)
model12.fit(X_iso, y_iso)


# Given an unbiased evaluation  (Question #E211)
# ----------------------------------
#confusion matrix, precision, recall, f1-score, accuracy
print("Parameters of the best model are:",model12.best_estimator_)
print("Score of the best model:", model12.best_score_)
predicted_y = model12.predict(X_test)
print(classification_report(y_test, predicted_y))
pd.DataFrame(confusion_matrix(y_test, predicted_y))

Parameters of the best model are: Pipeline(steps=[('scale', StandardScaler()), ('PCA', PCA(n_components=20)),
                ('LogisticRegression', LogisticRegression(C=10))])
Score of the best model: 0.7977224063109549
              precision    recall  f1-score   support

           0       0.89      0.72      0.79      3387
           1       0.76      0.91      0.83      3389

    accuracy                           0.81      6776
   macro avg       0.83      0.81      0.81      6776
weighted avg       0.83      0.81      0.81      6776



/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


,0,1
0,2435,952
1,308,3081


In [18]:
#model 13
#Anomaly detection: Elliptic Envelope, feature selection: Factor Analysis, Classifier: Logistic Regression
pipe = Pipeline([
    ('scale', StandardScaler()),
    ('FA', FactorAnalysis()),
    ('LogisticRegression', LogisticRegression())                 
])
# hyperparameter tunning for classifier and feature selector
param_grid = { 'FA__n_components': [15, 20],
               'LogisticRegression__C': [4, 5, 6, 9, 10]
              }

#model building and model fitting on train data
model13 = GridSearchCV(pipe, param_grid, cv=5, n_jobs=-1)
model13.fit(X_ee, y_ee)


#confusion matrix, precision, recall, f1-score, accuracy
print("Parameters of the best model are:",model13.best_estimator_)
print("Score of the best model:", model13.best_score_)
predicted_y = model13.predict(X_test)
print(classification_report(y_test, predicted_y))
pd.DataFrame(confusion_matrix(y_test, predicted_y))

Parameters of the best model are: Pipeline(steps=[('scale', StandardScaler()),
                ('FA', FactorAnalysis(n_components=20)),
                ('LogisticRegression', LogisticRegression(C=10))])
Score of the best model: 0.8206051853831523
              precision    recall  f1-score   support

           0       0.87      0.76      0.81      3387
           1       0.79      0.88      0.83      3389

    accuracy                           0.82      6776
   macro avg       0.83      0.82      0.82      6776
weighted avg       0.83      0.82      0.82      6776



,0,1
0,2587,800
1,394,2995


In [86]:
#model14
#Anomaly detection method: Isolation forest, feature selection method: factor analysis, Classifier: random forest
#defining pipeline
pipeline = Pipeline([
    ('scale', StandardScaler()),
    ('feature_selection', FactorAnalysis()),
    ('rf', RandomForestClassifier())
])

# define a grid of hyperparameters for the pipeline
param_grid = {
     'feature_selection__n_components': [15, 20],
     'rf__n_estimators': [50, 100, 1000],
     'rf__min_samples_split': [2, 3, 4],
     'rf__criterion': ['gini', 'entropy'] 
}

#model building and fitting model on train set
model14 = GridSearchCV(pipeline, param_grid, cv=5, n_jobs=-1)
model14.fit(X_iso, y_iso)
#confusion matrix, precision, recall, f1-score, accuracy

print("Parameters of the best model are:",model14.best_estimator_)
print("Score of the best model:", model14.best_score_)
predicted_y = model14.predict(X_test)
print(classification_report(y_test, predicted_y))
pd.DataFrame(confusion_matrix(y_test, predicted_y))

Parameters of the best model are: Pipeline(steps=[('scale', StandardScaler()),
                ('feature_selection', FactorAnalysis(n_components=20)),
                ('rf', RandomForestClassifier(n_estimators=1000))])
Score of the best model: 0.8652375365424634
              precision    recall  f1-score   support

           0       0.96      0.94      0.95      3395
           1       0.94      0.96      0.95      3381

    accuracy                           0.95      6776
   macro avg       0.95      0.95      0.95      6776
weighted avg       0.95      0.95      0.95      6776



,0,1
0,3186,209
1,124,3257


In [93]:
#model15

#Anomaly detection method: Elliptic Envelope, feature selection method: factor analysis, Classifier: random forest
#defining pipeline
pipeline = Pipeline([
    ('scale', StandardScaler()),
    ('feature_selection', FactorAnalysis()),
    ('rf', RandomForestClassifier())
])

# define a grid of hyperparameters for the pipeline
param_grid = {
     'feature_selection__n_components': [15, 20],
     'rf__n_estimators': [50, 100, 1000],
     'rf__min_samples_split': [2, 3, 4],
     'rf__criterion': ['gini', 'entropy'] 
}

#model building and fitting model on train set
model15 = GridSearchCV(pipeline, param_grid, cv=5, n_jobs=-1)
model15.fit(X_ee, y_ee)

#confusion matrix, precision, recall, f1-score, accuracy

print("Parameters of the best model are:",model15.best_estimator_)
print("Score of the best model:", model15.best_score_)
predicted_y = model15.predict(X_test)
print(classification_report(y_test, predicted_y))
pd.DataFrame(confusion_matrix(y_test, predicted_y))

Parameters of the best model are: Pipeline(steps=[('scale', StandardScaler()),
                ('feature_selection', FactorAnalysis(n_components=20)),
                ('rf', RandomForestClassifier(n_estimators=1000))])
Score of the best model: 0.8610056311216763
              precision    recall  f1-score   support

           0       0.88      0.81      0.84      3395
           1       0.82      0.89      0.85      3381

    accuracy                           0.85      6776
   macro avg       0.85      0.85      0.85      6776
weighted avg       0.85      0.85      0.85      6776



,0,1
0,2740,655
1,370,3011


In [92]:
#model16
#Anamoly detection method: Elliptic Envelope, feature selection: RFE, Classifier: Logistic Regression
#pipeline defination
pipe = Pipeline([
    ('scale', StandardScaler()),
    ('RFE', RFE(estimator=LogisticRegression())),
    ('LogisticRegression', LogisticRegression())                 
])
#parameters tunning
param_grid = { 'RFE__n_features_to_select': [10, 15],
               'LogisticRegression__C': [4, 5, 6, 9, 10]
              }

#model building
model16 = GridSearchCV(pipe, param_grid, cv=5, n_jobs=-1)
model16.fit(X_ee, y_ee)

# Given an unbiased evaluation  
# ----------------------------------
#confusion matrix, precision, recall, f1-score, accuracy
print("Parameters of the best model are:",model16.best_estimator_)
print("Score of the best model:", model16.best_score_)
predicted_y = model16.predict(X_test)
print(classification_report(y_test, predicted_y))
pd.DataFrame(confusion_matrix(y_test, predicted_y))
pd.DataFrame(confusion_matrix(y_test, predicted_y))

Parameters of the best model are: Pipeline(steps=[('scale', StandardScaler()),
                ('RFE',
                 RFE(estimator=LogisticRegression(), n_features_to_select=10)),
                ('LogisticRegression', LogisticRegression(C=9))])
Score of the best model: 0.8387108279139843
              precision    recall  f1-score   support

           0       0.89      0.80      0.84      3395
           1       0.82      0.90      0.86      3381

    accuracy                           0.85      6776
   macro avg       0.85      0.85      0.85      6776
weighted avg       0.85      0.85      0.85      6776



,0,1
0,2706,689
1,328,3053


In [91]:
#model17

# Anomaly detection method: Elliptic Envelope, Feature selection: Factor Analysis  Classifier: Logistic Regression

pipe = Pipeline([
    ('scale', StandardScaler()),
    ('FA', FactorAnalysis()),
    ('LogisticRegression', LogisticRegression())                 
])
# hyperparameter tunning for classifier and feature selector
param_grid = { 'FA__n_components': [15, 20],
               'LogisticRegression__C': [4, 5, 6, 9, 10]
              }

#model building and model fitting on train data
model17 = GridSearchCV(pipe, param_grid, cv=5, n_jobs=-1)
model17.fit(X_ee, y_ee)


print("Parameters of the best model are:",model17.best_estimator_)
print("Score of the best model:", model17.best_score_)
predicted_y = model17.predict(X_test)
print(classification_report(y_test, predicted_y))
print("Confusion Matrix")
pd.DataFrame(confusion_matrix(y_test, predicted_y))

Parameters of the best model are: Pipeline(steps=[('scale', StandardScaler()),
                ('FA', FactorAnalysis(n_components=15)),
                ('LogisticRegression', LogisticRegression(C=4))])
Score of the best model: 0.7743495296458317
              precision    recall  f1-score   support

           0       0.79      0.72      0.75      3395
           1       0.74      0.80      0.77      3381

    accuracy                           0.76      6776
   macro avg       0.76      0.76      0.76      6776
weighted avg       0.76      0.76      0.76      6776

Confusion Matrix


,0,1
0,2435,960
1,663,2718


In [ ]:
#modelx
#classifier: XGB, Feature selection method: RFE
#defining pipeline
pipeline = Pipeline([
    ('scale', StandardScaler()),
    ('RFE', RFE(estimator=XGBClassifier())),
    ('XGB', XGBClassifier())
])

# define a grid of hyperparameters for the pipeline
param_grid = {
    'RFE__n_features_to_select': [10, 15],
    'XGB__max_depth': [3, 7],
    'XGB__n_estimators': [50, 200],
    'XGB__learning_rate': [0.01, 0.1]
}

model4 = GridSearchCV(pipeline, param_grid, cv=5, n_jobs=-1)
model4.fit(X_svm, y_svm)


model_results4 = pd.DataFrame.from_dict(model4.cv_results_)
print(model_results4.sort_values("rank_test_score"))


# Given an unbiased evaluation  (Question #E211)
# ----------------------------------
print("Parameters of the best model are:",model4.best_estimator_)
print("Score of the best model:", model4.best_score_)
predicted_y = model4.predict(X_test)
print(classification_report(y_test, predicted_y))



# It took more than a hour to execute

## Comparison of these three pipelines and findings

### Pickle the required pipeline/models for Part III.

In [19]:
joblib.dump(bestmodel1, 'model1_pca_ksvc.pkl')


['model1_pca_ksvc.pkl']

In [20]:
joblib.dump(bestmodel2, 'model2_fa_lsvc.pkl')

['model2_fa_lsvc.pkl']

In [38]:
joblib.dump(bestmodel3, 'model3_rfe_rf.pkl')

['model3_rfe_rf.pkl']

In [26]:
# for dumping 
joblib.dump([iso_forest, bestmodel3], 'data_sample_data_v1.pkl')

['data_sample_data_v1.pkl']